<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load Data</a></span></li><li><span><a href="#Extract-Features-and-Targets" data-toc-modified-id="Extract-Features-and-Targets-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Extract Features and Targets</a></span></li><li><span><a href="#Create-Validation-Set" data-toc-modified-id="Create-Validation-Set-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Create Validation Set</a></span></li><li><span><a href="#Explore-Data" data-toc-modified-id="Explore-Data-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Explore Data</a></span></li><li><span><a href="#Manage-Missing-Categorical-Data" data-toc-modified-id="Manage-Missing-Categorical-Data-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Manage Missing Categorical Data</a></span></li><li><span><a href="#Manage-Missing-Numerical-Data" data-toc-modified-id="Manage-Missing-Numerical-Data-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Manage Missing Numerical Data</a></span><ul class="toc-item"><li><span><a href="#Drop-Numerical-Features-with-Missing-Data" data-toc-modified-id="Drop-Numerical-Features-with-Missing-Data-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Drop Numerical Features with Missing Data</a></span></li><li><span><a href="#Imputation" data-toc-modified-id="Imputation-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Imputation</a></span></li><li><span><a href="#Mixed-Dropping-and-Imputation" data-toc-modified-id="Mixed-Dropping-and-Imputation-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>Mixed Dropping and Imputation</a></span></li></ul></li><li><span><a href="#Scikit-Learn-Built-In-Imputer" data-toc-modified-id="Scikit-Learn-Built-In-Imputer-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Scikit Learn Built In Imputer</a></span></li><li><span><a href="#Drop-Categorical-Data" data-toc-modified-id="Drop-Categorical-Data-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Drop Categorical Data</a></span></li><li><span><a href="#Label-Encode-Categorical-Data" data-toc-modified-id="Label-Encode-Categorical-Data-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Label Encode Categorical Data</a></span></li><li><span><a href="#One-Hot-Encode-Categorical-Data" data-toc-modified-id="One-Hot-Encode-Categorical-Data-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>One-Hot Encode Categorical Data</a></span></li><li><span><a href="#Evaluate-Dropping-Categorical-Data" data-toc-modified-id="Evaluate-Dropping-Categorical-Data-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Evaluate Dropping Categorical Data</a></span></li></ul></div>

# Import Packages

In [1]:
import pandas as pd
import pandas_profiling as pp

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Import Data

## Load Data

In [2]:
# Define data locations
data_dir        = '../Data/house-prices-advanced-regression-techniques/'
train_file_name = 'train.csv'
test_file_name  = 'test.csv'

# Load training and testing data
train_data = pd.read_csv( data_dir + train_file_name, index_col='Id' )
test_data  = pd.read_csv( data_dir + test_file_name, index_col='Id' )

# Remove rows with missing targets
train_data.dropna( axis=0, subset=['SalePrice'], inplace=True)

## Extract Features and Targets

In [3]:
# Extract targets and features
y = train_data.SalePrice
X = train_data.copy()
X.drop( ['SalePrice'], axis=1, inplace=True )


X_test = test_data.copy()

# As instructed by course, use only numerical data
# Update: Include categorical data
# Uncomment if categorical data no longer wanted
#X = X.select_dtypes( exclude=['object'] )
#X_test = test_data.select_dtypes( exclude=['object'] )

## Create Validation Set

In [4]:
X_train, X_val, y_train, y_val = train_test_split( X, y, 
                                                   train_size=0.8, 
                                                   test_size=0.2, 
                                                   random_state=0)

## Explore Data

In [ ]:
'LotFrontage', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
       'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'GarageYrBlt',
       'GarageCars', 'GarageArea'],
      dtype='object')

In [41]:
pp.ProfileReport( X_val )

Number of variables,79
Number of observations,292
Total Missing (%),0.3%
Total size in memory,180.3 KiB
Average record size in memory,632.3 B
Numeric,36
Categorical,41
Boolean,1
Date,0
Text (Unique),0
Rejected,1


# Manage Missing Data

## Manage Missing Categorical Data

In [12]:
# Get the columns for categorical features that are missing values
cat_feats_missing_vals = list(X_train.columns[(X_train.dtypes =='object') & X_train.isnull().any()])
print( cat_feats_missing_vals )

['Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature']


Based on the provided description, most of the missing values should be replaced with "NA"

The only special cases are:
 
MasVnrType: Masonry veneer type
- None: None

BsmtExposure: Refers to walkout or garden level walls
- No:   No Exposure
- NA:   No Basement

Electrical: Electrical system
- No good label, as such we will remove this featue. Additionaly, from the Profile Report we see this feature has a uniqueness of 0.4% so this feature may not tell us much about the data anyway.

In [13]:
# Possible reasons for missing values on BsmtExposure could be "no exposure" or "no basement"
# We can do a simple crossreference to see how many times values exist in other basement categories and not for BsmtExposure
(X_train['BsmtExposure'].isnull() & X_train['BsmtCond'].notnull()).sum()

0

No times does this explanation occure, so we can safely assume missing values in BsmtExposure are due to "no basement" 

In [14]:
# Manage special cases in categorical data
X_train = X_train.fillna( value={'MasVnrType' : 'None', 'BsmtExposure' : 'NA'}, inplace=False )
X_val   = X_val.fillna( value={'MasVnrType' : 'None', 'BsmtExposure' : 'NA'}, inplace=False )
X_test  = X_test.fillna( value={'MasVnrType' : 'None', 'BsmtExposure' : 'NA'}, inplace=False )

X_train.drop( columns=['Electrical'], inplace=True )
X_val.drop( columns=['Electrical'], inplace=True )
X_test.drop( columns=['Electrical'], inplace=True )

In [15]:
# Manage all other cases of missing values in categorical data
value = {}
for feat in cat_feats_missing_vals:
    value[feat] = 'NA'
    
X_train = X_train.fillna( value=value, inplace=False )
X_val   = X_val.fillna( value=value, inplace=False )
X_test  = X_test.fillna( value=value, inplace=False )

In [17]:
pp.ProfileReport(X_train)

Number of variables,79
Number of observations,1168
Total Missing (%),0.3%
Total size in memory,721.0 KiB
Average record size in memory,632.1 B
Numeric,37
Categorical,42
Boolean,0
Date,0
Text (Unique),0
Rejected,0


## Manage Missing Numerical Data

### Drop Numerical Features with Missing Data

In [45]:
def drop_missing_numeric( Train=X_train, Test=X_val, feats_to_drop=[] ):
    # Features with missing values
    if not feats_to_drop:
        feats_to_drop = list(Train.columns[(Train.dtypes !='object') & Train.isna().any()])

    reduced_Train = Train.drop( feats_to_drop, axis=1, inplace=False )
    reduced_Test  = Test.drop( feats_to_drop, axis=1, inplace=False )
        
    return reduced_Train, reduced_Test

### Imputation

In [22]:
def impute_missing_numeric( replacement='median', Train=X_train, Test=X_val ):
    # Features with missing values
    feats_missing_vals = list(Train.columns[(Train.dtypes !='object') & Train.isna().any()])
    
    # Get replacement value given desired statistic
    if replacement == 'mean':
        replacement = Train[feats_missing_vals].mean( skipna=True )
    elif replacement == 'median':
        replacement = Train[feats_missing_vals].median( skipna=True )
    elif replacement == 'min':
        replacement = Train[feats_missing_vals].min( skipna=True )
    
    # Impute missing values with calculated replacement
    imputed_Train = Train.fillna( replacement )
    imputed_Test  = Test.fillna( replacement )

    return imputed_Train, imputed_Test

### Mixed Dropping and Imputation

Looking at the feature descriptions gives rise to intuition about whether removing the feature or imputation of the feature makes sense.

LotFrontage - Linear feet of street connected to property  
- Likely missing if no street is connected to property such as an apartment or condo.  
- If this is the case, it makes sense to use imputation with 0's to fill for NAN

MasVnrArea  - Masonry veneer area in square feet  
- Likely missing if no masonry veneer  
- If this is the case, it makes sense to use imputation with 0's to fill for NAN

GarageYrBlt - Year garage was built  
- Likely missing if no garage  
- If this is the case, imputation does not make much sense and simply removing the feature may result in better calssification

In [49]:
def mixed_drop_impute_missing_numeric( Train=X_train, Test=X_val ):
    # Drop year garage was built
    reduced_Train, reduced_Test = drop_missing_numeric( Train=Train, Test=Test, feats_to_drop=['GarageYrBlt'] )

    # Perform scalar imputation with 0's
    return impute_missing_numeric( replacement=0, Train=reduced_Train, Test=reduced_Test )

## Scikit Learn Built In Imputer

A less "reinventing the wheel" heavy method is to use Scikit Learn's built in simple imputer class.

I think I like the way I performed Imputation above better. There is less code involved and it seems to be simpler operations. It also has the benefit of working natively with Panda's Data Frame.

In [ ]:
#from sklearn.impute import SimpleImputer

#sklearn_imputed_X_train = X_train.copy()
#sklearn_imputed_X_val   = X_val.copy()
#median_imputer  = SimpleImputer( strategy='median' )

#sklearn_imputed_X_train = pd.DataFrame( median_imputer.fit_transform(sklearn_imputed_X_train) )
#sklearn_imputed_X_val   = pd.DataFrame( median_imputer.transform(sklearn_imputed_X_val) )

#sklearn_imputed_X_train.columns = X_train.columns
#sklearn_imputed_X_val.columns = X_val.columns

# Manage Categorical Data

## Drop Categorical Data

In [42]:
def drop_categorical_data( Train=X_train, Test=X_val ):
    reduced_Train = Train.select_dtypes( exclude=['object'] )
    reduced_Test  = Test.select_dtypes( exclude=['object'] )
    
    return reduced_Train, reduced_Test

## Label Encode Categorical Data

In [25]:
def label_encode_categorical_data( Train=X_train, Test=X_val ):
    # Initial setup
    categorical_feats = Train.columns[Train.dtypes == 'object']
    label_encoder = LabelEncoder()
    encoded_Train = Train.copy()
    encoded_Test  = Test.copy()
    
    # Find "good feats" and "bad feats" 
    # Bad feats are feats with values in the test set that are not in the training set.
    good_feats = [feat for feat in categorical_feats 
                  if set(Train[feat]) == set(Test[feat])]
    bad_feats  = categorical_feats.difference( good_feats )
        
    # Drop bad feats
    encoded_Train.drop( bad_feats, axis=1, inplace=True )
    encoded_Test.drop( bad_feats, axis=1, inplace=True )
    
    # Encode good feats
    for feat in good_feats:
        encoded_Train[feat] = label_encoder.fit_transform( Train[feat] )
        encoded_Test[feat]  = label_encoder.transform( Test[feat] )
        
    return encoded_Train, encoded_Test

## One-Hot Encode Categorical Data

In [26]:
def one_hot_encode_categorical_data( Train=X_train, Test=X_val, card_thresh=10, drop_high_card=False ):
    # Initial setup
    categorical_feats = Train.columns[Train.dtypes == 'object']
    one_hot_encoder = OneHotEncoder( handle_unknown='ignore', sparse=False )
    encoded_Train = Train.copy()
    encoded_Test  = Test.copy()
    
    # Apply cardinality threshold to categorical features
    low_card_feats  = [feat for feat in categorical_feats if Train[feat].nunique() <= card_thresh]
    high_card_feats = categorical_feats.difference( low_card_feats )    
        
    # Fit encoder on low cardinality features and apply to training and testing data
    one_hot_Train = pd.DataFrame( one_hot_encoder.fit_transform( Train[low_card_feats] ) )
    one_hot_Test  = pd.DataFrame( one_hot_encoder.transform( Test[low_card_feats] ) )
    
    # One-hot encoding returns a numpy array which removed the indices; put them back
    one_hot_Train.index = Train.index
    one_hot_Test.index  = Test.index
    
    # Remove low cardinality categorical columns and replace them with the one-hot encoded columns
    encoded_Train.drop( low_card_feats, axis=1, inplace=True )
    encoded_Test.drop( low_card_feats, axis=1, inplace=True )
    encoded_Train = pd.concat( [encoded_Train, one_hot_Train], axis=1 )
    encoded_Test  = pd.concat( [encoded_Test, one_hot_Test], axis=1 )
    
    # Manage high cardinality features
    if drop_high_card:
        encoded_Train.drop( high_card_feats, axis=1, inplace=True )
        encoded_Test.drop( high_card_feats, axis=1, inplace=True )
    else:
        encoded_Train, encoded_Test = label_encode_categorical_data( Train=encoded_Train,
                                                                     Test=encoded_Test )
    
    return encoded_Train, encoded_Test

# Model Creation

Defined via the specs on the Kaggle course.

In [27]:
model = RandomForestRegressor(n_estimators=100, random_state=0)

# Evaluate System

A system being the combination of model and dataset.

In [34]:
def score_system( m=model, X_t=X_train, X_v=X_val, y_t=y_train, y_v=y_val ):
    m.fit( X_t, y_t )
    pred_val = m.predict( X_v )
    return mean_absolute_error( pred_val, y_v )

In [46]:
def evaluate_missing_numerical_data_methods(  m=model, X_t=X_train, X_v=X_val, y_t=y_train, y_v=y_val ):
    print( 'MAE with dropping missing numerical data:' )
    drop_X_t, drop_X_v = drop_missing_numeric( Train=X_t, Test=X_v )
    print( score_system( X_t=drop_X_t, X_v=drop_X_v ) )
    
    print( 'MAE with mean imputing numerical data:' )
    mean_impute_X_t, mean_impute_X_v = impute_missing_numeric( replacement='mean', Train=X_t, Test=X_v )
    print( score_system( X_t=mean_impute_X_t, X_v=mean_impute_X_v ) )
    
    print( 'MAE with median imputing numerical data:' )
    median_impute_X_t, median_impute_X_v = impute_missing_numeric( replacement='median', Train=X_t, Test=X_v )
    print( score_system( X_t=median_impute_X_t, X_v=median_impute_X_v ) )
    
    print( 'MAE with min imputing numerical data:' )
    min_impute_X_t, min_impute_X_v = impute_missing_numeric( replacement='min', Train=X_t, Test=X_v )
    print( score_system( X_t=min_impute_X_t, X_v=min_impute_X_v ) )
    
    print( 'MAE with mixed imputing and dropping numerical data:' )
    mixed_impute_X_t, mixed_impute_X_v = mixed_drop_impute_missing_numeric( Train=X_t, Test=X_v )
    print( score_system( X_t=mixed_impute_X_t, X_v=mixed_impute_X_v ) )

## Evaluate Dropping Categorical Data 

In [47]:
drop_cat_X_train, drop_cat_X_val = drop_categorical_data()

In [51]:
evaluate_missing_numerical_data_methods( X_t=drop_cat_X_train, X_v=drop_cat_X_val )

MAE with dropping missing numerical data:
17837.82570776256
MAE with mean imputing numerical data:
18062.894611872147
MAE with median imputing numerical data:
17791.59899543379
MAE with min imputing numerical data:
18079.878230593607
MAE with mixed imputing and dropping numerical data:
18133.380205479454
